In [7]:
import plotly.express as px

# Load the Gapminder dataset
gap_data = px.data.gapminder()

In [ ]:
import pandas as pd
import numpy as np

trade_mx = pd.read_csv(r"..\data\trade_matrix_carbon_footprint_v4.csv")
codes = pd.read_csv(r"..\data\country_matrix_v4_final.csv")

codes['key'] = codes['country_from_code'].astype(str) + '-' + codes['country_to_code'].astype(str)
trade_mx['key'] = trade_mx['Reporter Country Code'].astype(str) + '-' + trade_mx['Partner Country Code'].astype(str)

In [10]:
full_trade_mx = trade_mx.merge(codes[["country_from", "continent_from", "key"]], on='key', how='left')

In [14]:
full_trade_mx.head()

,Year,country_from,continent_from,kg_CO2eq_tonne-km
0,2005,algeria,africa,175.660885
1,2005,albania,europe,1130.462293
2,2022,angola,africa,1903.799176
3,1996,algeria,africa,10012.670425
4,1997,algeria,africa,9368.580515


In [13]:
cols_to_keep = ['Year', 'country_from', 'continent_from', 'kg_CO2eq_tonne-km']
full_trade_mx = full_trade_mx[cols_to_keep]

In [19]:
co2_mx = full_trade_mx.groupby(['country_from', 'continent_from', 'Year'])['kg_CO2eq_tonne-km'].sum().to_frame().reset_index()

In [23]:
# Convert the 'country' column in df2 to lowercase
gap_data['country'] = gap_data['country'].str.lower()

# Merge the dataframes on 'country' and 'year'
co2_mx_merged = pd.merge(co2_mx, gap_data, left_on=['country_from', 'Year'], right_on=['country', 'year'])

# Drop the duplicate columns
co2_mx_merged.drop(columns=['country', 'year'], inplace=True)

In [24]:
co2_mx_merged

,country_from,continent_from,Year,kg_CO2eq_tonne-km,continent,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,albania,europe,1997,6.200736e+07,Europe,72.950,3428038,3193.054604,ALB,8
1,albania,europe,2002,3.028127e+08,Europe,75.651,3508512,4604.211737,ALB,8
2,albania,europe,2007,3.583407e+08,Europe,76.423,3600523,5937.029526,ALB,8
3,algeria,africa,1987,2.214205e+08,Africa,65.799,23254956,5681.358539,DZA,12
4,algeria,africa,1992,4.667653e+08,Africa,67.744,26298373,5023.216647,DZA,12
...,...,...,...,...,...,...,...,...,...,...
392,zambia,africa,2002,7.862133e+07,Africa,39.193,10595811,1071.613938,ZMB,894
393,zambia,africa,2007,4.976818e+07,Africa,42.384,11746035,1271.211593,ZMB,894
394,zimbabwe,africa,1992,1.767716e+08,Africa,60.377,10704340,693.420786,ZWE,716
395,zimbabwe,africa,1997,5.400112e+07,Africa,46.809,11404948,792.449960,ZWE,716


In [27]:
# Create a moving bubble chart
fig = px.scatter(co2_mx_merged.sort_values(by='Year'),  
                 #x="gdpPercap", 
                 y="kg_CO2eq_tonne-km", 
                 size="pop", 
                 color="continent_from", 
                 hover_name="country_from", 
                 log_x=True,
                 log_y=True, 
                 size_max=60, 
                 animation_frame="Year", 
                 animation_group="country_from",
                 title="GDP per Capita vs kg CO2eq produced from food import")

# Update layout for better visualization
fig.update_layout(xaxis_title='GDP per Capita (log scale)',
                  yaxis_title='kg CO2eq (log scale)',
                  legend_title='Continent',
                  title_font_size=24)

# Show the figure
fig.show()
